[View in Colaboratory](https://colab.research.google.com/github/abunickabhi/IRproject/blob/master/Word_embedding.ipynb)

In [26]:
!git clone https://github.com/abunickabhi/IRproject.git ; rm -rf embed/ ;mkdir embed ;cd IRproject/ ; git pull origin
!pip3 install Cython
!pip3 install word2vec
!pip3 install fasttext
!pip3 install gensim
!pip3 install torch
!pip3 install PyDictionary


fatal: destination path 'IRproject' already exists and is not an empty directory.
remote: Counting objects: 3, done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/abunickabhi/IRproject
   e43a63e..03485aa  master     -> origin/master
Updating e43a63e..03485aa
Fast-forward
 Embedding_Reference.pdf | Bin 0 -> 634800 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Embedding_Reference.pdf


**Importing all the required libraries**

In [0]:
import gensim
import word2vec
import fasttext
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
#from src.dictionary import Dictionary
from PyDictionary import PyDictionary
from torch import optim




**Sync the dataset in the Google Drive into the notebook environment**

In [28]:
'''from google.colab import files
import io

a=files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
df = pd.read_csv(io.StringIO(uploaded['english.vec'].decode('utf-8')))
print(df)
'''

'from google.colab import files\nimport io\n\na=files.upload()\nfor fn in uploaded.keys():\n  print(\'User uploaded file "{name}" with length {length} bytes\'.format(name=fn, length=len(uploaded[fn])))\ndf = pd.read_csv(io.StringIO(uploaded[\'english.vec\'].decode(\'utf-8\')))\nprint(df)\n'

**Attempting to sync the dataset in the Google Drive by FUSE**

In [29]:
'''# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print 'Files in Drive:'
!ls drive/

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt
'''

'# Install a Drive FUSE wrapper.\n# https://github.com/astrada/google-drive-ocamlfuse\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\n\n# Generate auth tokens for Colab\nfrom google.colab import auth\nauth.authenticate_user()\n\n# Generate creds for the Drive FUSE library.\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n\n# Create a directory and mount Google Drive using that directory.\n!mkdir -p drive\n!google-drive-ocamlfuse drive\n\nprint \'Files in Drive:\'\n!ls dri

**Loading and Processing the source words i.e. English**



In [30]:
max_vocab = 10000
emb_dim = 300
src_lang = 'eng'

src_word2id = {}
src_embeddings = []
a= 'IRproject/reduced_English.vec'

with open(a) as f:
    for i,line in enumerate(f):
        if i==0:
            split = line.split()
            assert len(split) == 2
            assert emb_dim == int(split[1])
        else:
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            if np.linalg.norm(vect)==0: #to avoid null embeddings
                vect[0] = 0.01
            assert word not in src_word2id
            assert vect.shape == (emb_dim, )
            src_word2id[word] = len(src_word2id)
            src_embeddings.append(vect[None,:])
        if i > max_vocab:
            break
            
src_id2word = {v: k for k,v in src_word2id.items()}
src_embeddings = np.concatenate(src_embeddings,0)
src_embeddings = torch.from_numpy(src_embeddings).float()
src_emb = nn.Embedding(len(src_word2id), emb_dim, sparse=True)
src_emb.weight.data.copy_(src_embeddings)


 0.1073  0.0089  0.0006  ...   0.0050  0.1173 -0.0400
 0.0897  0.0160 -0.0571  ...   0.1559 -0.0254 -0.0259
 0.0004  0.0032 -0.0204  ...   0.2070  0.0689 -0.0467
          ...             ⋱             ...          
-0.0675  0.0383 -0.0183  ...  -0.3375  0.0463 -0.0164
 0.0806 -0.0063  0.0875  ...   0.2192  0.0188 -0.0608
-0.0583  0.0550 -0.0072  ...  -0.2309 -0.0549  0.0410
[torch.FloatTensor of size 13x300]

**Loading the source language i.e. Tamil into a Dictionary**

In [31]:
#src_dico = Dictionary(src_id2word, src_word2id, src_lang)

tgt_lang = 'ta'
tgt_word2id = {}
tgt_embeddings = []
b= 'IRproject/reduced_Tamil.vec'

with open(b) as f:
    for i,line in enumerate(f):
        if i==0:
            split = line.split()
            assert len(split) == 2
            assert emb_dim == int(split[1])
        else:
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            if np.linalg.norm(vect)==0: #to avoid null embeddings
                vect[0] = 0.01
            assert word not in tgt_word2id
            assert vect.shape == (emb_dim, )
            tgt_word2id[word] = len(tgt_word2id)
            tgt_embeddings.append(vect[None,:])
        if i > max_vocab:
            break

tgt_id2word = {v:k for k,v in tgt_word2id.items()}

#tgt_dico = PyDictionary(tgt_id2word, tgt_word2id, tgt_lang)
tgt_embeddings = np.concatenate(tgt_embeddings,0)
tgt_embeddings = torch.from_numpy(tgt_embeddings).float()
tgt_emb = nn.Embedding(len(tgt_word2id), emb_dim, sparse=True)
tgt_emb.weight.data.copy_(tgt_embeddings)

mapping = nn.Linear(emb_dim, emb_dim, bias=False)
mapping.weight.data.copy_(torch.diag(torch.ones(emb_dim)))




    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.FloatTensor of size 300x300]

**Create a self class in Discriminator class and define the parameters of training**

In [32]:
disc_layers = 2
disc_dim_hidden = 2048
disc_dropout = 0
disc_inp_dropout = 0.1

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.emb_dim = emb_dim
        self.disc_layers = disc_layers
        self.disc_dim_hidden = disc_dim_hidden
        self.disc_dropout = disc_dropout
        self.disc_inp_dropout = disc_inp_dropout
        
        layers = [nn.Dropout(self.disc_inp_dropout)]
        for i in range(self.disc_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.disc_dim_hidden
            output_dim = 1 if i==self.disc_layers else self.disc_dim_hidden
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.disc_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.disc_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x).view(-1)
      
discriminator = Discriminator()
'''
src_emb.cuda()
tgt_emb.cuda()
mapping.cuda()
discriminator.cuda()
'''

'\nsrc_emb.cuda()\ntgt_emb.cuda()\nmapping.cuda()\ndiscriminator.cuda()\n'

**Use Stochastic Gradient Optimizer from Torch optim library to get source and target prediction and accuracy score**

****

In [0]:
optim_fn = optim.SGD
optim_params = {'lr': 0.1}

map_optimizer = optim_fn(mapping.parameters(), **optim_params)
disc_optimizer = optim_fn(discriminator.layers.parameters(), **optim_params)
epoch_size = 1000000
batch_size = 32
disc_steps = 5
disc_most_freq = 75000
disc_smooth = 0.1
map_beta = 0.001

def get_disc_xy(volatile):
    """
    Get discriminator input batch / output target.
    """
    # select random word IDs
    bs = batch_size
    mf = disc_most_freq
    assert mf <= min(len(src_dico), len(tgt_dico))
    src_ids = torch.LongTensor(bs).random_(mf)
    tgt_ids = torch.LongTensor(bs).random_(mf)
    src_ids = src_ids.cuda()
    tgt_ids = tgt_ids.cuda()

    # get word embeddings
    src_emb = src_emb(Variable(src_ids, volatile=True))
    tgt_emb = tgt_emb(Variable(tgt_ids, volatile=True))
    src_emb = mapping(Variable(src_emb.data, volatile=volatile))
    tgt_emb = Variable(tgt_emb.data, volatile=volatile)

    # input / target
    x = torch.cat([src_emb, tgt_emb], 0)
    y = torch.FloatTensor(2 * bs).zero_()
    y[:bs] = 1 - disc_smooth
    y[bs:] = dis_smooth
    y = Variable(y.cuda())

    return x, y
  
def orthogonalize():
  
    W = mapping.weight.data
    beta = map_beta
    W.copy_((1 + beta) * W - beta * W.mm(W.transpose(0, 1).mm(W)))
    
 
def eval_dis(self, to_log):
  
    bs = 128
    src_preds = []
    tgt_preds = []

    discriminator.eval()

    for i in range(0, src_emb.num_embeddings, bs):
        emb = Variable(src_emb.weight[i:i + bs].data, volatile=True)
        preds = discriminator(mapping(emb))
        src_preds.extend(preds.data.cpu().tolist())

    for i in range(0,tgt_emb.num_embeddings, bs):
        emb = Variable(tgt_emb.weight[i:i + bs].data, volatile=True)
        preds = discriminator(emb)
        tgt_preds.extend(preds.data.cpu().tolist())

    src_pred = np.mean(src_preds)
    tgt_pred = np.mean(tgt_preds)
    print("Discriminator source / target predictions: %.5f / %.5f"
                % (src_pred, tgt_pred))

    src_accu = np.mean([x >= 0.5 for x in src_preds])
    tgt_accu = np.mean([x < 0.5 for x in tgt_preds])
    dis_accu = ((src_accu * src_emb.num_embeddings + tgt_accu * tgt_emb.num_embeddings) /
                    (src_emb.num_embeddings + tgt_emb.num_embeddings))
    print("Discriminator source / target / global accuracy: %.5f / %.5f / %.5f"
                    % (src_accu, tgt_accu, dis_accu))

**Adversarial Training**

In [34]:
def adv_training(epochs):
    for epoch in range(epochs):
        n_words_proc = 0
        for n_iter in range(0,epoch_size, batch_size):
            for _ in range(disc_steps): # Discriminator training
                discriminator.train() # sets the module in training mode ex adds dropout and batchnorm
                x,y = get_disc_xy(volatile=True)
                preds = discriminator(Variable(x.data))
                loss = F.binary_cross_entropy(preds,y)
                disc_optimizer.zero_grad()
                loss.backward()
                disc_optimizer.step()
                # Can add clipping if needed
            
            #Mapping step
            discriminator.eval() # Puts the module in evaluation mode.
            x, y = get_disc_xy(volatile=False)
            preds = discriminator(Variable(x.data))
            loss = F.binary_cross_entropy(preds, 1-y)
            map_optimizer.zero_grad()
            loss.backward()
            map_optmizer.step()
            orthogonalize()
            n_words_proc += 2 * batch_size
            
        # embeddings / discriminator evaluation
        eval_dis()


exp_path = ""
def export_embeddings(src_emb, tgt_emb, params):
    
    src_id2word = src_dico.id2word
    tgt_id2word = tgt_dico.id2word
    n_src = len(src_id2word)
    n_tgt = len(tgt_id2word)
    dim = src_emb.shape[1]
    src_path = os.path.join(exp_path, 'vectors-%s.txt' % src_lang)
    tgt_path = os.path.join(exp_path, 'vectors-%s.txt' % tgt_lang)
    # source embeddings
    with open(src_path, 'w') as f:
        f.write("%i %i\n" % (n_src, dim))
        for i in range(len(src_id2word)):
            f.write("%s %s\n" % (src_id2word[i], " ".join(str(x) for x in src_emb[i])))
   # target embeddings
    with open(tgt_path, 'w') as f:
        f.write("%i %i\n" % (n_tgt, dim))
        for i in range(len(tgt_id2word)):
            f.write("%s %s\n" % (tgt_id2word[i], " ".join(str(x) for x in tgt_emb[i])))
'''          
src_emb = mapping(src_emb.weight).data
tgt_emb = tgt_emb.weight.data
src_emb = src_emb / src_emb.norm(2, 1, keepdim=True).expand_as(src_emb)
tgt_emb = tgt_emb / tgt_emb.norm(2, 1, keepdim=True).expand_as(tgt_emb)
export_embeddings(src_emb.cpu().numpy(), tgt_emb.cpu().numpy(), self.params)
'''

'          \nsrc_emb = mapping(src_emb.weight).data\ntgt_emb = tgt_emb.weight.data\nsrc_emb = src_emb / src_emb.norm(2, 1, keepdim=True).expand_as(src_emb)\ntgt_emb = tgt_emb / tgt_emb.norm(2, 1, keepdim=True).expand_as(tgt_emb)\nexport_embeddings(src_emb.cpu().numpy(), tgt_emb.cpu().numpy(), self.params)\n'